# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas 
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [5]:
#Load the CSV file created in Part 1 into a Pandas DataFrame
city_data = pd.read_csv("C:/Users/dreww/Desktop/DataAnalysisWork/Assignments/Starter_Code (2)/Starter_Code/output_data/cities.csv")
city_data_df = pd.DataFrame(city_data)
# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mil'kovo,55.6078,37.7956,0.22,94,100,1.92,RU,1698626986
1,1,iqaluit,63.7506,-68.5145,-8.15,73,100,5.14,CA,1698626986
2,2,lapeer,43.0514,-83.3188,8.21,76,100,1.54,US,1698626986
3,3,tiksi,71.6872,128.8694,-16.28,98,98,3.05,RU,1698626987
4,4,cooma,-36.2333,149.1333,22.94,25,100,8.33,AU,1698626987


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
#Configure Plot
data = {
    "City": city_data_df["City"],
    "Latitude": city_data_df["Lat"],
    "Longitude": city_data_df["Lng"],
    "Humidity": city_data_df["Humidity"]
}

df = pd.DataFrame(data)
plot = df.hvplot.points(
    geo=True,
    x="Longitude",
    y="Latitude",
    size="Humidity",
    colorbar=True,
    hover_cols=["City", "Humidity"]
).opts(
    cmap="coolwarm",
    tools=["hover"]
)

#Show Plot
plot


:Points   [Longitude,Latitude]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()


# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,bhadasar,28.2833,74.3167,24.83,33,0,2.29,IN,1698626991
77,77,cambria,35.5641,-121.0808,21.54,64,0,0.45,US,1698626996
95,95,fort bragg,35.1390,-79.0060,21.75,78,0,0.00,US,1698626998
112,112,santa maria,-29.6842,-53.8069,21.61,88,0,2.06,BR,1698626646
114,114,salinas,36.6777,-121.6555,23.20,62,0,2.06,US,1698626956


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
34,bhadasar,IN,28.2833,74.3167,33,
77,cambria,US,35.5641,-121.0808,64,
95,fort bragg,US,35.1390,-79.0060,78,
112,santa maria,BR,-29.6842,-53.8069,88,
114,salinas,US,36.6777,-121.6555,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = # YOUR CODE HERE
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

SyntaxError: invalid syntax (788381735.py, line 2)

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)